In [1]:
import pyrealsense2 as rs2
import open3d as o3d
import numpy as np
import time
from spygrt.stream import Recording, NoMoreFrames, DualRecording
from spygrt.tracker import Tracker
from spygrt.calibration import Calibrator
import importlib
import time

In [2]:
#SET UP

rec1 = 'E:/VISION/134/Recording_Session/121622061907_2023Apr12013010.bag'
rec2 = 'E:/VISION/134/Recording_Session/123122061443_2023Apr12013010.bag'
cal1 = 'E:/VISION/134/RS_Calibration/121622061907_2023Apr12010202.bag'
cal2 = 'E:/VISION/134/RS_Calibration/123122061443_2023Apr12010202.bag'
opt1 = 'E:/VISION/134/OptiTrack_Calibration_Square/121622061907_2023Apr12011406.bag'
opt2 = 'E:/VISION/134/OptiTrack_Calibration_Square/123122061443_2023Apr12011406.bag'
d = DualRecording(rec1,rec2)
d.start_stream()
c = DualRecording(cal1,cal2)
c.start_stream()
calibs = (Calibrator(c.stream1), Calibrator(c.stream2))
o = DualRecording(opt1,opt2)
o.start_stream()

In [4]:
# CALIBRATION FOR ESTRO

for i in range (100):
    try:
        calibs[0].find_corners3d()
        calibs[1].find_corners3d()
    except NoMoreFrames:
        print("Reached the end of stream, used only the first " +str(i) + " frames for calibration.")
        break

#calibs[0].corners_scale()
#calibs[1].corners_scale()
calibs[0].align_cameras(calibs[1])
calibs[1].align_to_board(cal2 = calibs[0])
for cal in calibs:
    cal.write_cal()
    cal._pose = o3d.core.Tensor(np.identity(4), dtype=o3d.core.Dtype.Float32,device=cal._stream.pcd.device)
c.load_calibration()
for i in range (100):
    try:
        calibs[0].find_corners3d()
        calibs[1].find_corners3d()
    except NoMoreFrames:
        print("Reached the end of stream, used only the first " +str(i) + " frames for calibration.")
        break

#calibs[0].corners_scale()
#calibs[1].corners_scale()
calibs[0].align_cameras(calibs[1])
calibs[1].align_to_board(cal2 = calibs[0])
adj = o3d.core.Tensor(np.identity(4), dtype=o3d.core.Dtype.Float32, device=cal._pose.device)
adj[1][1]=-1
adj[0][0]=-1
adj[0][3] = -0.0215
adj[1][3] = -0.060
adj[2][3] = -0.120
calibs[0]._pose = adj@calibs[0].pose
calibs[1]._pose = adj@calibs[1].pose
for cal in calibs:
    cal.write_cal()

In [3]:
t = Tracker(ref_surface=d.pcd)
data = []
import time
a = time.time()
for i in range(120000):
    
    try:
        t.get_motion(d.compute_pcd(True))
    except RuntimeError:
        continue
    except NoMoreFrames:
        np.savetxt('track_134.txt',data)
        break
    nt = t.t_guess.inv()
    data.append(np.concatenate(([d.timestamp], Tracker.compute_translation(nt), Tracker.compute_rotation(nt))))
    if i%100 == 0:
        print(i/100)
        np.savetxt('track_134.txt',data)
b = time.time()
b-a   

[Open3D INFO] Picked point #30425 (-0.089, -0.057, 0.047) to add in queue.
[Open3D INFO] Picked point #596031 (0.031, -0.18, 0.08) to add in queue.
Timestamps:1681270210372.857 1681270210665.8003
Timestamps:1681270210406.4495 1681270210665.8003
Timestamps:1681270210440.0417 1681270210665.8003
Timestamps:1681270210473.6338 1681270210665.8003
Timestamps:1681270210507.226 1681270210665.8003
Timestamps:1681270210540.8184 1681270210665.8003
0.0
Timestamps:1681270211750.2732 1681270211875.1367
Timestamps:1681270213060.728 1681270213218.9492
Timestamps:1681270213060.728 1681270213218.9492
Timestamps:1681270213060.728 1681270213218.9492
Timestamps:1681270213060.728 1681270213218.9492
Timestamps:1681270213060.728 1681270213218.9492
Timestamps:1681270213060.728 1681270213218.9492
Timestamps:1681270213060.728 1681270213218.9492
Timestamps:1681270213060.728 1681270213218.9492
Timestamps:1681270213060.728 1681270213218.9492
Timestamps:1681270213060.728 1681270213218.9492
Timestamps:1681270213060.72

Timestamps:1681270231976.0354 1681270232167.2317
Timestamps:1681270232043.2239 1681270232167.2317
Timestamps:1681270232345.5715 1681270232503.5005
Timestamps:1681270232345.5715 1681270232503.5005
Timestamps:1681270233185.7244 1681270233343.7227
Timestamps:1681270233690.1226 1681270233578.955
Timestamps:1681270234059.8044 1681270233914.9612
Timestamps:1681270234059.8044 1681270233948.5603
Timestamps:1681270234059.8044 1681270233948.5603
Timestamps:1681270235369.9666 1681270234452.5525
Timestamps:1681270235369.9666 1681270234452.5525
3.0
Timestamps:1681270235638.7656 1681270235527.652
Timestamps:1681270235840.3672 1681270235729.2275
Timestamps:1681270235840.3672 1681270235729.2275
Timestamps:1681270235840.3672 1681270235729.2275
Timestamps:1681270235840.3672 1681270235997.9912
Timestamps:1681270236209.98 1681270236098.7776
Timestamps:1681270236209.98 1681270236333.9448
Timestamps:1681270236209.98 1681270236333.9448
Timestamps:1681270236613.1968 1681270236737.0925
Timestamps:1681270237788

Timestamps:1681270260702.418 1681270259918.829
Timestamps:1681270260702.418 1681270259918.829
Timestamps:1681270260702.418 1681270259918.829
Timestamps:1681270260702.418 1681270259918.829
Timestamps:1681270260702.418 1681270259918.829
Timestamps:1681270260702.418 1681270259918.829
Timestamps:1681270260702.418 1681270259918.829
Timestamps:1681270260702.418 1681270259918.829
Timestamps:1681270260702.418 1681270259918.829
Timestamps:1681270260702.418 1681270259918.829
Timestamps:1681270260702.418 1681270259918.829
Timestamps:1681270260702.418 1681270259918.829
Timestamps:1681270260702.418 1681270259918.829
Timestamps:1681270260702.418 1681270259918.829
Timestamps:1681270260702.418 1681270259918.829
Timestamps:1681270260702.418 1681270259918.829
Timestamps:1681270260702.418 1681270259918.829
Timestamps:1681270261038.3816 1681270260893.0588
Timestamps:1681270261038.3816 1681270260893.0588
Timestamps:1681270261710.2827 1681270261833.642
Timestamps:1681270262012.6365 1681270262169.584
Timesta

Timestamps:1681270286874.81 1681270287031.8215
Timestamps:1681270287210.7922 1681270287099.011
Timestamps:1681270287210.7922 1681270287334.1736
Timestamps:1681270287210.7922 1681270287334.1736
Timestamps:1681270288319.576 1681270288442.8306
Timestamps:1681270288386.771 1681270288510.0244
Timestamps:1681270289125.9119 1681270289013.9763
Timestamps:1681270289461.8752 1681270289316.3384
Timestamps:1681270289529.0676 1681270289417.1255
Timestamps:1681270290335.4075 1681270289618.6995
Timestamps:1681270290335.4075 1681270289618.6995
Timestamps:1681270290335.4075 1681270289618.6995
Timestamps:1681270291007.414 1681270290895.8157
Timestamps:1681270291007.414 1681270290895.8157
Timestamps:1681270291007.414 1681270291131.0142
Timestamps:1681270291074.6143 1681270291198.214
Timestamps:1681270291612.1685 1681270291769.419
Timestamps:1681270292418.531 1681270292542.1143
Timestamps:1681270292418.531 1681270292542.1143
Timestamps:1681270292888.9395 1681270292777.2727
10.0
Timestamps:1681270293258.51

Timestamps:1681270320774.9075 1681270320662.5312
Timestamps:1681270320774.9075 1681270320897.7134
Timestamps:1681270320774.9075 1681270320897.7134
Timestamps:1681270321043.6895 1681270320931.3108
Timestamps:1681270321043.6895 1681270320931.3108
Timestamps:1681270321043.6895 1681270320931.3108
Timestamps:1681270321043.6895 1681270320931.3108
Timestamps:1681270321043.6895 1681270320931.3108
Timestamps:1681270321043.6895 1681270320931.3108
Timestamps:1681270321043.6895 1681270321368.0596
Timestamps:1681270321043.6895 1681270321368.0596
Timestamps:1681270321110.885 1681270321368.0596
Timestamps:1681270321110.885 1681270321368.0596
Timestamps:1681270321178.0806 1681270321368.0596
Timestamps:1681270321211.679 1681270321368.0596
Timestamps:1681270321245.2773 1681270321368.0596
Timestamps:1681270321245.2773 1681270321368.0596
Timestamps:1681270322186.0383 1681270322308.7683
Timestamps:1681270322186.0383 1681270322308.7683
Timestamps:1681270322421.2168 1681270322308.7683
Timestamps:168127032242

...

Timestamps:1681270860893.2227 1681270859996.6782
Timestamps:1681270860893.2227 1681270859996.6782
Timestamps:1681270860893.2227 1681270859996.6782
Timestamps:1681270860893.2227 1681270859996.6782
Timestamps:1681270861800.592 1681270861979.1553
Timestamps:1681270862170.234 1681270861979.1553
Timestamps:1681270862170.234 1681270862012.7546
Timestamps:1681270862170.234 1681270862046.354
Timestamps:1681270862237.435 1681270862113.5527
Timestamps:1681270862674.2483 1681270862785.5518
Timestamps:1681270862909.459 1681270862785.5518
Timestamps:1681270863077.4595 1681270862953.5366
Timestamps:1681270863144.6526 1681270863020.726
Timestamps:1681270863413.4248 1681270863255.8892
Timestamps:1681270863413.4248 1681270863255.8892
Timestamps:1681270863413.4248 1681270863255.8892
Timestamps:1681270863413.4248 1681270863524.6458
Timestamps:1681270863883.774 1681270864028.5652
Timestamps:1681270863917.37 1681270864028.5652
Timestamps:1681270863917.37 1681270864028.5652
Timestamps:1681270865194.4956 168

Timestamps:1681270878699.7932 1681270878576.8425
Timestamps:1681270880312.3916 1681270880491.7422
Timestamps:1681270880312.3916 1681270880491.7422
Timestamps:1681270882765.4822 1681270882641.9534
Timestamps:1681270883135.0234 1681270882944.2742
Timestamps:1681270883135.0234 1681270882977.8677
Timestamps:1681270883135.0234 1681270882977.8677
Timestamps:1681270883605.4543 1681270883481.7676
Timestamps:1681270884378.2644 1681270884254.4524
Timestamps:1681270884479.0503 1681270884355.2466
Timestamps:1681270884479.0503 1681270884355.2466
Timestamps:1681270884479.0503 1681270884590.4329
Timestamps:1681270884546.2407 1681270884657.629
82.0
Timestamps:1681270884848.5967 1681270884960.011
Timestamps:1681270885117.357 1681270884993.6433
Timestamps:1681270885117.357 1681270884993.6433
Timestamps:1681270885117.357 1681270884993.6433
Timestamps:1681270885117.357 1681270885229.079
Timestamps:1681270885117.357 1681270885229.079
Timestamps:1681270885554.2712 1681270885430.8896
Timestamps:1681270885554

Timestamps:1681270895633.1855 1681270895509.1528
Timestamps:1681270895834.752 1681270895677.22
Timestamps:1681270895834.752 1681270895677.22
Timestamps:1681270896069.9114 1681270895946.1316
Timestamps:1681270896137.0996 1681270896013.3562
Timestamps:1681270897514.767 1681270897390.5464
Timestamps:1681270897514.767 1681270897390.5464
Timestamps:1681270897514.767 1681270897390.5464
Timestamps:1681270897514.767 1681270897390.5464
Timestamps:1681270897548.368 1681270897692.9055
Timestamps:1681270897548.368 1681270897692.9055
Timestamps:1681270899161.1226 1681270898196.813
Timestamps:1681270899161.1226 1681270898196.813
Timestamps:1681270899161.1226 1681270898196.813
Timestamps:1681270899597.997 1681270899708.3418
Timestamps:1681270899597.997 1681270899708.3418
Timestamps:1681270899833.2454 1681270899708.3418
Timestamps:1681270899967.6738 1681270900111.414
Timestamps:1681270900270.1157 1681270900145.0063
Timestamps:1681270900908.3914 1681270900783.2556
Timestamps:1681270901513.1394 16812709

Timestamps:1681270912667.3499 1681270912509.1714
Timestamps:1681270912667.3499 1681270912542.7634
85.0
Timestamps:1681270913003.3274 1681270912811.497
Timestamps:1681270913003.3274 1681270912878.6802
Timestamps:1681270913003.3274 1681270912878.6802
Timestamps:1681270913541.0469 1681270913651.2876
Timestamps:1681270914280.6548 1681270914087.9727
Timestamps:1681270914549.4143 1681270914659.16
Timestamps:1681270914784.5737 1681270914894.3586
Timestamps:1681270914784.5737 1681270914894.3586
Timestamps:1681270915187.7017 1681270915028.758
Timestamps:1681270915187.7017 1681270915028.758
Timestamps:1681270915187.7017 1681270915028.758
Timestamps:1681270915187.7017 1681270915028.758
Timestamps:1681270915187.7017 1681270915028.758
Timestamps:1681270915187.7017 1681270915028.758
Timestamps:1681270915187.7017 1681270915028.758
Timestamps:1681270915187.7017 1681270915028.758
Timestamps:1681270915187.7017 1681270915028.758
Timestamps:1681270915187.7017 1681270915028.758
Timestamps:1681270915187.701

2562.5253937244415

In [4]:
o.load_calibration(force=True)

(array([[ 0.73145878, -0.46030766,  0.50307554, -0.38821465],
        [-0.68160343, -0.51479334,  0.52000433, -0.41811669],
        [ 0.01961797, -0.72325975, -0.69029737,  0.48424962],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[ 0.75355726,  0.47636428, -0.4530215 ,  0.41423345],
        [ 0.6517849 , -0.4516702 ,  0.60923761, -0.44040293],
        [ 0.08560272, -0.75436801, -0.65084642,  0.49168023],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]))

In [4]:
t = Tracker(ref_surface=d.pcd)

[Open3D INFO] Picked point #216382 (-0.078, -0.033, 0.0016) to add in queue.
[Open3D INFO] Picked point #677477 (0.081, -0.18, 0.068) to add in queue.


In [10]:
ot = Tracker(ref_surface=o.pcd)

[Open3D INFO] Picked point #848194 (-0.0089, 0.0095, -0.12) to add in queue.
[Open3D INFO] Picked point #461669 (0.0090, -0.075, -0.093) to add in queue.


In [6]:
for cal in calibs:
    cal.write_cal()

In [ ]:
[Open3D INFO] Picked point #1061769 (-0.037, 0.21, 0.023) to add in queue.
[Open3D INFO] Picked point #220126 (-0.028, 0.015, 0.027) to add in queue.
[Open3D INFO] Picked point #1023532 (-0.18, 0.013, 0.029) to add in queue.

In [7]:
data[1]

array([ 1.68127298e+12, -1.02986378e-04,  2.80050254e-05, -5.62721025e-05,
       -2.75960183e-02, -3.15382275e-02,  4.29738733e-02])